<a href="https://colab.research.google.com/github/laNaranjaja/GenAI/blob/main/Streamlit_CV_Screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q streamlit
! pip install Openai
! pip install langchain
! pip install replicate
! pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the p

In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.598s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Aqui va el codigo de la aplicacion

%%writefile app.py
import streamlit as st
import os
from langchain.chat_models import ChatOpenAI
from langchain.llms import Replicate
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models import ChatVertexAI
from pypdf import PdfReader
import json

st.write("CV Screening App :sunglasses:")

with st.sidebar:
  openai_api_key_file = st.file_uploader('Sube aqui tu key de OpenAI', type=['txt'])
  replicate_api_key_file = st.file_uploader('Sube aqui tu key de Replicate', type=['txt'])

if (openai_api_key_file is not None) & (replicate_api_key_file is not None):

  #Modelo OpenAI GPT 3.5
  openai_key = str(openai_api_key_file.readline().decode('utf-8'))
  os.environ['OPENAI_API_KEY'] = openai_key
  chat_openai = ChatOpenAI(model = 'gpt-3.5-turbo-16k')

  #Modelo Replicate LLama2 70b
  replicate_key = str(replicate_api_key_file.readline().decode('utf-8'))
  os.environ['REPLICATE_API_TOKEN'] = replicate_key
  chat_replicate = Replicate(
    model='meta/llama-2-70b-chat:2d19859030ff705a87c746f7e96eea03aefb71f166725aee39692f1476566d48',
    model_kwargs={'temperature':0.7, 'max_length':3500}
    )

  #Modelo PALM2 text-bison
  os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/gdrive/MyDrive/GenAIEne2024/unified-surface-410604-c94c40d752ce.json'
  chat_gcp = ChatVertexAI(max_output_tokens=2048, temperature=0)

  modelo = st.selectbox(
    'Que modelo desea utilizar para esta tarea?',
    ('GPT 3.5 Turbo', 'Llama2', 'PALM 2'))


  archivo_pdf = st.file_uploader('Sube aqui tu CV en formato PDF', type=['pdf'])

  if (archivo_pdf is not None):
    pdfreader = PdfReader(archivo_pdf)
    text = ''
    for page in pdfreader.pages:
      text += page.extract_text()

    if modelo=='GPT 3.5 Turbo':
      prompt = """
      Eres un reclutador experimentado de recursos humanos.
      Tu trabajo consiste en revisar curriculumns de diversos candidatos
      Deberas entregar un resumen que incluya:
      1) El nombre del candidato
      2) Tres experiencias laborales mas recientes mencionando la empresa, el rol y el año
      3) Ultimos 2 grados de estudio mencionando la escuela, el prograsma y el año
      4) Certificaciones enlistadas en una tabla de 3 columnas
      5) Habilidades enlistadas en una tabla de 2 columnas
      6) 3 Ventajas y 3 desventajas de considerar a ese candidato con base en la informacion proporcionada.
      Deberas usar emojis para cada seccion del resumen.
      TODO el resumen debera estar en español. No dejar nada en inglés
      """
      query=text

      with st.spinner('Generando respuesta con GPT 3.5 Turbo...'):
          result = chat_openai(
          [
              SystemMessage(content=prompt),
              HumanMessage(content=query)
          ])
          st.write(result.content)
          st.success('Respuesta generada exitosamente!', icon="✅")
          st.balloons()

    elif modelo=='Llama2':
      prompt = f"""
      Eres un reclutador experimentado de recursos humanos.
      Tu trabajo consiste en revisar curriculumns de diversos candidatos
      Deberas entregar un resumen que incluya:
      1) El nombre del candidato
      2) Tres experiencias laborales mas recientes mencionando la empresa, el rol y el año
      3) Ultimos 2 grados de estudio mencionando la escuela, el prograsma y el año
      4) Certificaciones y/o habilidades extra enlistadas en una lista de 3 columnas
      5) 3 Ventajas y 3 desventajas de considerar a ese candidato con base en la informacion proporcionada.
      Deberas usar emojis para cada seccion del resumen.
      TODO el resumen debera estar en español. No dejar nada en inglés
      Datos del candidato: {text}
      """

      with st.spinner('Generando respuesta con Llama2...'):
        resultado = chat_replicate(prompt)
        st.write(resultado)
        st.success('Respuesta generada exitosamente!', icon="✅")
        st.balloons()

    elif modelo=='PALM 2':
      prompt = """
      Eres un reclutador experimentado de recursos humanos.
      Tu trabajo consiste en revisar curriculumns de diversos candidatos
      Deberas entregar un resumen que incluya:
      1) El nombre del candidato
      2) Tres experiencias laborales mas recientes mencionando la empresa, el rol y el año
      3) Ultimos 2 grados de estudio mencionando la escuela, el prograsma y el año
      4) Certificaciones enlistadas en una tabla de 3 columnas
      5) Habilidades enlistadas en una tabla de 2 columnas
      6) 3 Ventajas y 3 desventajas de considerar a ese candidato con base en la informacion proporcionada.
      Deberas usar emojis para cada seccion del resumen.
      TODO el resumen debera estar en español. No dejar nada en inglés
      """
      query=text

      with st.spinner('Generando respuesta con PALM 2...'):
          result = chat_gcp(
          [
              SystemMessage(content=prompt),
              HumanMessage(content=query)
          ])
          st.write(result.content)
          st.success('Respuesta generada exitosamente!', icon="✅")
          st.balloons()



Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.125.131.180


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.368s
your url is: https://real-moments-stay.loca.lt
^C
